In [1]:
import pandas as pd
import os

# specify the folder containing the CSV files
folder = 'Telangana weather'

# specify the name of the new CSV file
new_file = 'adilabad.csv'

# list to hold the data frames from each CSV file
df_list = []

# list of month names in the correct order
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September']

# loop through each month name
for month in months:
    # construct the file name for the CSV file for that month
    file = f"TS Weather data {month} 2022.csv"
    file_path = os.path.join(folder, file)
    # check if the file exists
    if os.path.exists(file_path):
        print("yes")
        # read the CSV file into a data frame
        df = pd.read_csv(file_path)
        # select only the rows where the district is 'Adilabad' and the mandal is 'Adilabad rural'
        df = df[(df['District'] == 'Adilabad') & (df['Mandal'] == 'Adilabad Rural')]
        # add the filtered data frame to the list
        df_list.append(df)

# concatenate all the data frames into a single data frame
result = pd.concat(df_list)

# write the result to a new CSV file
result.to_csv(new_file, index=False)

yes
yes
yes
yes
yes
yes
yes
yes


In [2]:
import pandas as pd

# Read the "adilabad.csv" file
df = pd.read_csv('adilabad.csv')

# Calculate the mean temperature and add it as a new column
df['Mean Temp (°C)'] = (df['Max Temp (°C)'] + df['Min Temp (°C)']) / 2

# Write the updated data frame to a new CSV file
df.to_csv('adilabad_mean_temp.csv', index=False)

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import *

# Load the data into a pandas DataFrame
df = pd.read_csv('adilabad_mean_temp.csv')

# Normalize the temperature values using MinMaxScaler
temp_cols = ['Min Temp (°C)', 'Max Temp (°C)', 'Mean Temp (°C)']
scaler = MinMaxScaler(feature_range=(0, 1))
df[temp_cols] = scaler.fit_transform(df[temp_cols])

# Convert the DataFrame into a numpy array for input into the LSTM model
data = df[temp_cols].values

# Define the number of time steps and number of features
timesteps = 100
n_features = 3

# Create the input and output data for the LSTM model
X, y = [], []
for i in range(timesteps, len(data)):
    X.append(data[i-timesteps:i, :])
    y.append(data[i, :])
X, y = np.array(X), np.array(y)

# Split the data into training and test sets
train_size = int(len(X) * 0.9)
X_train, X_test = X[:train_size, :, :], X[train_size:, :, :]
y_train, y_test = y[:train_size, :], y[train_size:, :]

# Define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(timesteps, n_features)))
model.add(Dense(n_features))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the LSTM model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

# Use the trained LSTM model to make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions to the original scale
y_pred_scaled = scaler.inverse_transform(y_pred)
y_test_scaled = scaler.inverse_transform(y_test)
# Calculate the R-squared score for mean temperature
r2 = r2_score(y_test_scaled[:, 2], y_pred_scaled[:, 2])
print("R-squared:", r2)

1/1 [==============================] - 0s 266ms/step
R-squared: -0.30592347103941875


In [4]:
 #Convert the dates in the original DataFrame to a datetime format
df['Date'] = pd.to_datetime(df['Date'])
print(df['Date'].dtype)
# Create a DataFrame to store the actual and predicted values along with the corresponding date
results = pd.DataFrame({'Date': df['Date'][-len(y_test_scaled):], 
                        'Actual Min Temp (°C)': y_test_scaled[:, 0], 
                        'Actual Max Temp (°C)': y_test_scaled[:, 1], 
                        'Actual Mean Temp (°C)': y_test_scaled[:, 2], 
                        'Predicted Min Temp (°C)': y_pred_scaled[:, 0], 
                        'Predicted Max Temp (°C)': y_pred_scaled[:, 1], 
                        'Predicted Mean Temp (°C)': y_pred_scaled[:, 2]})

datetime64[ns]


In [5]:
results['Date'] = results['Date'].astype(str)
# Save the predictions to a CSV file
results.to_csv('pred.csv', index=False)

In [6]:
print(results['Date'].dtype)

object


In [7]:
# Create a DataFrame to store the actual and predicted values
df_pred = pd.DataFrame(y_test_scaled, columns=temp_cols)
df_pred['Min Temp (°C) Pred'] = y_pred_scaled[:, 0]
df_pred['Max Temp (°C) Pred'] = y_pred_scaled[:, 1]
df_pred['Mean Temp (°C) Pred'] = y_pred_scaled[:, 2]

# Save the predictions to a CSV file
df_pred.to_csv('pred.csv', index=False)

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

# Load the data from the "pred.csv" file into a pandas DataFrame
pred = pd.read_csv("pred.csv")

# Extract the actual and predicted values for the "Max Temp (°C)" column
actual = pred['Max Temp (°C)'].values
predicted = pred['Max Temp (°C) Pred'].values

# Calculate the root mean squared error (RMSE)
mse = mean_squared_error(actual, predicted)
rmse = np.sqrt(mse)
print("RMSE:", rmse)



RMSE: 1.2364136019911254


In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

# Load the data from the "pred.csv" file into a pandas DataFrame
pred = pd.read_csv("pred.csv")

# Extract the actual and predicted values for the "Max Temp (°C)" column
actual = pred['Min Temp (°C)'].values
predicted = pred['Min Temp (°C) Pred'].values

# Calculate the root mean squared error (RMSE)
mse = mean_squared_error(actual, predicted)
rmse = np.sqrt(mse)
print("RMSE:", rmse)


RMSE: 1.1833096363002646


In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

# Load the data from the "pred.csv" file into a pandas DataFrame
pred = pd.read_csv("pred.csv")

# Extract the actual and predicted values for the "Max Temp (°C)" column
actual = pred['Mean Temp (°C)'].values
predicted = pred['Mean Temp (°C) Pred'].values

# Calculate the root mean squared error (RMSE)
mse = mean_squared_error(actual, predicted)
rmse = np.sqrt(mse)
print("RMSE:", rmse)


RMSE: 0.775307745326203
